In [0]:
df = (
    spark.read
         .option("header", "true")
         .option("inferSchema", "true")
         .csv("/Volumes/workspace/default/dataqcdataset/ecommerce_transactions.csv")
)

df.show(5)



+--------------+---------------+---+-------+----------------+---------------+--------------+----------------+
|Transaction_ID|      User_Name|Age|Country|Product_Category|Purchase_Amount|Payment_Method|Transaction_Date|
+--------------+---------------+---+-------+----------------+---------------+--------------+----------------+
|             1|       Ava Hall| 63| Mexico|        Clothing|         780.69|    Debit Card|      2023-04-14|
|             2|    Sophia Hall| 59|  India|          Beauty|         738.56|        PayPal|      2023-07-30|
|             3|Elijah Thompson| 26| France|           Books|         178.34|   Credit Card|      2023-09-17|
|             4|   Elijah White| 43| Mexico|          Sports|         401.09|           UPI|      2023-06-21|
|             5|     Ava Harris| 48|Germany|          Beauty|         594.83|   Net Banking|      2024-10-29|
+--------------+---------------+---+-------+----------------+---------------+--------------+----------------+
only showi

In [0]:
df.printSchema()
print("Total transactions:", df.count())


root
 |-- Transaction_ID: integer (nullable = true)
 |-- User_Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Country: string (nullable = true)
 |-- Product_Category: string (nullable = true)
 |-- Purchase_Amount: double (nullable = true)
 |-- Payment_Method: string (nullable = true)
 |-- Transaction_Date: date (nullable = true)

Total transactions: 50000


In [0]:
from pyspark.sql.functions import when, lit

noisy_df = df.withColumn(
    "Purchase_Amount",
    when(df.Transaction_ID % 10 == 0, -100)   # every 10th row invalid
    .otherwise(df.Purchase_Amount)
)


In [0]:
from pyspark.sql.functions import col

invalid_purchase = noisy_df.filter(col("Purchase_Amount") <= 0).count()
print("Invalid Purchase_Amount rows:", invalid_purchase)


Invalid Purchase_Amount rows: 5000


In [0]:
# Age validation
invalid_age = df.filter((col("Age") < 18) | (col("Age") > 100)).count()
print("Invalid age rows:", invalid_age)

# High-value purchase check
high_value = df.filter(col("Purchase_Amount") > 10000).count()
print("High value transactions:", high_value)


Invalid age rows: 0
High value transactions: 0


In [0]:
clean_df = (
    noisy_df
    .dropna()
    .filter(col("Purchase_Amount") > 0)
)

print("Rows after cleaning:", clean_df.count())


Rows after cleaning: 45000


In [0]:
from pyspark.sql.functions import sum, count

analytics_df = (
    clean_df
    .groupBy("Product_Category")
    .agg(
        sum("Purchase_Amount").alias("total_revenue"),
        count("Transaction_ID").alias("total_transactions")
    )
)

analytics_df.show()


+----------------+------------------+------------------+
|Product_Category|     total_revenue|total_transactions|
+----------------+------------------+------------------+
|  Home & Kitchen| 2792994.600000002|              5570|
|        Clothing|        2865484.93|              5622|
|            Toys| 2881727.289999998|              5787|
|           Books|        2856499.34|              5620|
|          Sports|2866251.3100000005|              5667|
|          Beauty|2769292.1899999985|              5497|
|         Grocery|2788687.9099999974|              5545|
|     Electronics|2817001.5300000007|              5692|
+----------------+------------------+------------------+



In [0]:
clean_df.createOrReplaceTempView("transactions")


In [0]:
%sql
SELECT 
    Product_Category,
    SUM(Purchase_Amount) AS revenue,
    COUNT(Transaction_ID) AS transactions
FROM transactions
GROUP BY Product_Category
ORDER BY revenue DESC;


Product_Category,revenue,transactions
Toys,2881727.289999998,5787
Sports,2866251.3100000005,5667
Clothing,2865484.93,5622
Books,2856499.34,5620
Electronics,2817001.5300000007,5692
Home & Kitchen,2792994.600000002,5570
Grocery,2788687.9099999974,5545
Beauty,2769292.1899999985,5497


In [0]:
top_category = analytics_df.orderBy(col("total_revenue").desc()).first()

print("📊 BUSINESS INSIGHT")
print("Top Product Category:", top_category["Product_Category"])
print("Total Revenue:", top_category["total_revenue"])
print("Total Transactions:", top_category["total_transactions"])


📊 BUSINESS INSIGHT
Top Product Category: Toys
Total Revenue: 2881727.289999998
Total Transactions: 5787


In [0]:
from pyspark.sql.functions import avg

avg_spend = clean_df.groupBy("Country") \
    .agg(avg("Purchase_Amount").alias("avg_spend")) \
    .orderBy(col("avg_spend").desc())

avg_spend.show(5)


+---------+------------------+
|  Country|         avg_spend|
+---------+------------------+
|      USA| 512.1911958946895|
|   France| 508.8794694605439|
|   Brazil| 506.8329726088905|
|Australia|503.25277022509476|
|   Canada| 501.8651902587516|
+---------+------------------+
only showing top 5 rows


In [0]:
%sql
SELECT 
    Product_Category,
    SUM(Purchase_Amount) AS total_revenue
FROM transactions
GROUP BY Product_Category;


Product_Category,total_revenue
Home & Kitchen,2792994.600000002
Clothing,2865484.93
Toys,2881727.289999998
Books,2856499.34
Sports,2866251.3100000005
Beauty,2769292.1899999985
Grocery,2788687.9099999974
Electronics,2817001.5300000007


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    Country,
    COUNT(Transaction_ID) AS transactions
FROM transactions
GROUP BY Country
ORDER BY transactions DESC
LIMIT 10;


Country,transactions
Canada,4599
Mexico,4577
Germany,4528
India,4509
Australia,4487
France,4486
USA,4482
Brazil,4454
UK,4452
Japan,4426


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    Payment_Method,
    COUNT(*) AS usage_count
FROM transactions
GROUP BY Payment_Method;


Payment_Method,usage_count
Debit Card,7506
Net Banking,7331
Credit Card,7505
Cash on Delivery,7593
PayPal,7422
UPI,7643


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    CASE
        WHEN Age < 25 THEN 'Under 25'
        WHEN Age BETWEEN 25 AND 40 THEN '25–40'
        ELSE '40+'
    END AS age_group,
    AVG(Purchase_Amount) AS avg_spend
FROM transactions
GROUP BY age_group;


age_group,avg_spend
Under 25,505.49348417829196
40+,502.27655335036354
25–40,503.4891789135051


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    Transaction_Date,
    COUNT(Transaction_ID) AS daily_transactions
FROM transactions
GROUP BY Transaction_Date
ORDER BY Transaction_Date;


Transaction_Date,daily_transactions
2023-03-09,57
2023-03-10,67
2023-03-11,65
2023-03-12,67
2023-03-13,59
2023-03-14,68
2023-03-15,56
2023-03-16,48
2023-03-17,49
2023-03-18,58


Databricks visualization. Run in Databricks to view.

In [0]:
%sql
SELECT 
    COUNT(*) AS high_value_transactions
FROM transactions
WHERE Purchase_Amount > 500;


high_value_transactions
22687


In [0]:
# Always register views at the top
clean_df.createOrReplaceTempView("transactions")



In [0]:
%sql
SELECT
    spend_tier,
    COUNT(DISTINCT User_name) AS user_count
FROM (
    SELECT
        User_name,
        CASE
            WHEN SUM(Purchase_Amount) < 20 THEN 'Low Spender'
            WHEN SUM(Purchase_Amount) BETWEEN 25 AND 100 THEN 'Medium Spender'
            ELSE 'High Spender'
        END AS spend_tier
    FROM transactions
    GROUP BY User_name
)
GROUP BY spend_tier;


spend_tier,user_count
High Spender,100


In [0]:
%sql
SELECT
    Country,
    Product_Category,
    SUM(Purchase_Amount) AS revenue
FROM transactions
GROUP BY Country, Product_Category;


Country,Product_Category,revenue
Japan,Beauty,288901.6399999999
Germany,Clothing,278979.17000000004
Australia,Home & Kitchen,264819.24000000005
Japan,Clothing,272242.7200000001
Brazil,Clothing,299796.99999999994
Canada,Books,313580.14999999997
France,Sports,278220.8899999999
France,Clothing,296153.67000000004
Brazil,Beauty,275601.33999999997
India,Sports,302566.7


In [0]:
%sql
SELECT
    age_group,
    Payment_Method,
    COUNT(*) AS usage_count
FROM (
    SELECT *,
        CASE
            WHEN Age < 25 THEN 'Under 25'
            WHEN Age BETWEEN 25 AND 40 THEN '25–40'
            WHEN Age BETWEEN 41 AND 60 THEN '41–60'
            ELSE '60+'
        END AS age_group
    FROM transactions
)
GROUP BY age_group, Payment_Method
ORDER BY age_group, usage_count DESC;


age_group,Payment_Method,usage_count
25–40,Debit Card,2332
25–40,UPI,2329
25–40,Cash on Delivery,2312
25–40,PayPal,2259
25–40,Net Banking,2225
25–40,Credit Card,2220
41–60,Credit Card,2897
41–60,UPI,2880
41–60,Cash on Delivery,2840
41–60,PayPal,2815


In [0]:
from pyspark.sql.functions import count, avg

freq_spend_df = (
    clean_df.groupBy("User_name")
            .agg(
                count("Transaction_ID").alias("purchase_frequency"),
                avg("Purchase_Amount").alias("avg_spend")
            )
)

freq_spend_df.show(10)


+------------------+------------------+------------------+
|         User_name|purchase_frequency|         avg_spend|
+------------------+------------------+------------------+
|      Oliver White|               444| 509.0416891891891|
|       James Lewis|               503|496.01689860834983|
|         Ava Lewis|               438|  514.129497716895|
|   Olivia Anderson|               462|  475.427380952381|
|     Elijah Walker|               425| 509.9480470588235|
|       Oliver Hall|               452| 513.5534070796459|
|Isabella Rodriguez|               470| 510.5754468085106|
|      Elijah White|               404| 497.7911386138614|
|     Emma Anderson|               470|508.29623404255335|
|     Emma Thompson|               439| 495.3196583143509|
+------------------+------------------+------------------+
only showing top 10 rows


In [0]:
%sql
SELECT
    CASE
        WHEN dayofweek(Transaction_Date) IN (1,7) THEN 'Weekend'
        ELSE 'Weekday'
    END AS day_type,
    AVG(Purchase_Amount) AS avg_spend,
    COUNT(*) AS transactions
FROM transactions
GROUP BY day_type;


day_type,avg_spend,transactions
Weekday,501.9897617636973,32069
Weekend,505.7326912071767,12931


In [0]:
%sql
SELECT
    CASE
        WHEN purchase_count = 100 THEN 'One-time'
        WHEN purchase_count BETWEEN 120 AND 500 THEN 'Repeat'
        ELSE 'Loyal'
    END AS customer_type,
    COUNT(*) AS users
FROM (
    SELECT User_name, COUNT(*) AS purchase_count
    FROM transactions
    GROUP BY User_name
)
GROUP BY customer_type;


customer_type,users
Repeat,96
Loyal,4
